## Assignment 4 Code: Nursery Rhymes

In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.utils as ku

from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential

In [170]:
titles = []
text = [] # a list of nursery rhymes (referred to as "rhymes")

with open('nursery_rhymes.txt') as f:
    current = ""
    while True:
        line = f.readline()
        if not line: # end of file
            if current != "": text.append(current.lower().rstrip())
            break

        if not line.strip(): # skip blank lines
            continue
        if line.isupper(): # titles are all uppercase letters
            if current != "":
                text.append(current.lower().rstrip()) # add the previous rhyme to the list
                current = ""
            titles.append(line.strip()) 
            continue
        current += line.lstrip()

f.close()

In [171]:
data = pd.DataFrame({'titles': titles, 'text': text})
data.head()

,titles,text
0,THE QUEEN OF HEARTS,"the queen of hearts she made some tarts,\nall ..."
1,SAINT SWITHIN'S DAY,"st. swithin's day, if thou dost rain,\nfor for..."
2,DANCE TO YOUR DADDIE,"dance to your daddie,\nmy bonnie laddie,\ndanc..."
3,THE MAN IN THE MOON,the man in the moon\ncame tumbling down\nand a...
4,SIMPLE SIMON,"simple simon met a pie-man,\ngoing to the fair..."


In [172]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'to': 4,
 'he': 5,
 'i': 6,
 'in': 7,
 'was': 8,
 'little': 9,
 'she': 10,
 'you': 11,
 'of': 12,
 'my': 13,
 'with': 14,
 'his': 15,
 'that': 16,
 'it': 17,
 'for': 18,
 'all': 19,
 'as': 20,
 'on': 21,
 'is': 22,
 'her': 23,
 'they': 24,
 'will': 25,
 'when': 26,
 'up': 27,
 'there': 28,
 'so': 29,
 'went': 30,
 'not': 31,
 'do': 32,
 'them': 33,
 'old': 34,
 'have': 35,
 'him': 36,
 'be': 37,
 'had': 38,
 'said': 39,
 'me': 40,
 'what': 41,
 'says': 42,
 'but': 43,
 'then': 44,
 'down': 45,
 'your': 46,
 'one': 47,
 "won't": 48,
 'shall': 49,
 'dog': 50,
 'came': 51,
 'if': 52,
 'come': 53,
 'oh': 54,
 'man': 55,
 'jack': 56,
 'are': 57,
 'rowley': 58,
 'sing': 59,
 "i'll": 60,
 'away': 61,
 'out': 62,
 'go': 63,
 'made': 64,
 'were': 65,
 'cat': 66,
 'by': 67,
 'at': 68,
 'two': 69,
 'pig': 70,
 'day': 71,
 'three': 72,
 'this': 73,
 'get': 74,
 'would': 75,
 'who': 76,
 'an': 77,
 'lady': 78,
 'some': 79,
 'ho': 80,
 'very': 81,
 'over': 82,
 'house

In [173]:
# converts each nursery rhyme into a sequence of tokens
# refers back to previous words only within each rhyme
input_sequences = [] 
for rhyme in text:
    token_list = tokenizer.texts_to_sequences([rhyme])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [174]:
# sequence padding
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,
                                         maxlen=max_sequence_len, 
                                         padding='pre'))

In [175]:
# creates the predictors and labels
    # predictor --> N-grams sequence
    # label --> next word of the N-gram
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=vocab_size)

In [176]:
input_len = max_sequence_len - 1

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=input_len))
model.add(LSTM(150))
model.add(Dropout(0.1))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam')
model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
494/494 [==============================] - 187s 375ms/step - loss: 6.4795
Epoch 2/100
494/494 [==============================] - 178s 360ms/step - loss: 6.1191
Epoch 3/100
494/494 [==============================] - 177s 359ms/step - loss: 5.9403
Epoch 4/100
494/494 [==============================] - 178s 360ms/step - loss: 5.7667
Epoch 5/100
494/494 [==============================] - 178s 360ms/step - loss: 5.5981
Epoch 6/100
494/494 [==============================] - 178s 360ms/step - loss: 5.4174
Epoch 7/100
494/494 [==============================] - 178s 360ms/step - loss: 5.2222
Epoch 8/100
494/494 [==============================] - 178s 361ms/step - loss: 5.0292
Epoch 9/100
494/494 [==============================] - 178s 359ms/step - loss: 4.8427
Epoch 10/100
494/494 [==============================] - 178s 360ms/step - loss: 4.6573
Epoch 11/100
494/494 [==============================] - 177s 359ms/step - loss: 4.4779
Epoch 12/100
494/494 [==============================

In [183]:
import random
new_rhyme = ""

for i in range(30): # 30 lines
    for j in range(20): # 20 words
        if len(new_rhyme) == 0: # starts new rhyme with a randomly selected word 
            random_word = random.choice(list(word_index.keys()))
            new_rhyme += random_word + " "
            continue
        # translates the words in the new rhyme into a sequence of tokens
        token_list = tokenizer.texts_to_sequences([new_rhyme])[0] 
        token_list = pad_sequences([token_list], maxlen= 
                                max_sequence_len-1, padding='pre')
        preds = model.predict(token_list)
        predicted_word = np.argmax(preds, axis=1)

        next_word = ""
        for word, index in word_index.items():
            # converts the next word back from its token state
            if index == predicted_word:
                next_word = word
                break
        new_rhyme += next_word + " "
    new_rhyme += "\n"

1/1 [==============================] - 0s 47ms/step


In [184]:
print(new_rhyme)

with open('new_rhyme.txt', 'w') as f: 
    f.write(new_rhyme)
f.close()

well i was a little boy i washed my mammy's dishes i put my finger in my eye and pulled 
out golden fishes all beat the little dog right and all the way he ate the parritch bee night a 
crown with the wind he said it the wind said the floor with the barn he can it the king 
in the floor and drove the chimney soot and away the chimney best said and all the night he said 
so night the other blue oh the de town i is but may the farmer's sparrow to the little one 
i said you shall you be the little kittens ones said you shall i will make a fine wife you 
will you shall be make a wife home his little boy i said you shall i will not twinkle little 
star boys and girls and the night you the sun is the little one i will you be the little 
boy i will you be get the stew i make me a stew i said it a mother cock i 
will not get my dee home to a wife i shall you will the little one to you you too 
get the little boy and the little boy i shall we going for a little boy i said he were 
't and i will